In [1]:
from brian2 import *

In [2]:
# size of the network
NE          = 1000              # number of E cells 
NI          = 250               # number of I cells 

# simulation-related parameters  
simtime     = 5000.0*ms   	# total simulation time [ms] 
dt          = 0.1*ms     	# simulation step length [ms]
simulation_clock=Clock(dt=dt)   # clock for the time steps

# pyramidal cells
Cm_e        = 0.5*nF  		# [nF] total capacitance
gl_e        = 25.0*nS 		# [ns] total leak conductance
El_e        = -70.0*mV		# [mV] leak reversal potential
Vth_e       = -50.0*mV		# [mV] threshold potential
Vr_e	    = -60.0*mV		# [mV] resting potential
tr_e	    = 2.0*ms		# [ms] refractory time

# interneuron cells
Cm_i        = 0.2*nF		# [nF] total capacitance
gl_i        = 20.0*nS		# [ns] total leak conductance
El_i        = -70.0*mV 		# [mV] leak reversal potential
Vth_i       = -50.0*mV 		# [mV] threshold potential
Vr_i	    = -60.0*mV 		# [mV] resting potential
tr_i	    = 1.0*ms  		# [ms] refractory time

# external background input
fext	    = 1800.0*Hz		# [Hz] external input frequency (poisson train)

# AMPA receptor (APMAR)
E_ampa 	    = 0.0*mV  		# [mV] synaptic reversial potential
t_ampa	    = 2.0*ms  		# [ms] exponential decay time constant 
g_ext_e	    = 3.1*nS  		# [nS] maximum conductance from external to pyramidal cells
g_ext_i	    = 2.38*nS 		# [nS] maximum conductance from external to interneuron cells

# GABA receptor (GABAR)
E_gaba 	    = -70.0*mV	  	# [mV] synaptic reversial potential
t_gaba	    = 10.0*ms		# [ms] exponential decay time constant 
Gie	    = 1.336*512/NI*nS #1.34765625*nS          # [ns] synaptic conductance interneuron to pyramidal cells
Gii	    = 1.024*512/NI*nS #1.024*nS          # [ns] synaptic conductance interneuron to interneuron cells

# NMDA receptor (NMDAR)
E_nmda 	    = 0.0*mV	 	# [mV] synaptic reversial potential
t_nmda      = 100.0*ms		# [ms] decay time of NMDA currents
t_x	    = 2.0*ms		# [ms] controls the rise time of NMDAR channels
alfa        = 0.5*kHz	  	# [kHz] controls the saturation properties of NMDAR channels
a           = 0.062/mV          # [1/mV] control the voltage dependance of NMDAR channel
b           = 1.0/3.57          # [1] control the voltage dependance of NMDAR channel ([Mg2+]=1mM )
Gee	    = 0.381*2048/NE*nS #0.381*nS          # [ns] synaptic conductance from pyramidal to pyramidal cells
Gei	    = 0.292*2048/NE*nS #0.292*nS          # [ns] synaptic conductance from pyramidal to interneuron cells

In [3]:
from scipy.special import erf

# Connection parameters
Jp_ee 	    = 1.62	  	# maximum connection weight from pyramidal to pyramidal
sigma_ee    = 14.4	 	# width of the connectivity footprint from pyramidal to pyramidal

# connectivity footprint
tmp   = sqrt(2*pi)*sigma_ee*erf(360.*0.5/sqrt(2.)/sigma_ee)/360.
Jm_ee = (1.-Jp_ee*tmp)/(1.-tmp)

weight=lambda i:(Jm_ee+(Jp_ee-Jm_ee)*exp(-0.5*(360.*min(i,NE-i)/NE)**2/sigma_ee**2))

weight_e=zeros(NE)
for i in range(NE): 
    weight_e[i]=weight(i)

#weight_e/=sum(weight_e)

fweight = rfft(weight_e) # Fourier transform

In [4]:
# cue parameters

tc_start      = 500*ms           # start of the cue signal
tc_dur        = 250*ms            # length period of the cue signal
i_cue_amp     = 200*pA            # cue current amplitude
i_cue_ang     = 180               # mean angle of the cue current
i_cue_width   = 18.0              # sigma/width of the current signal in degrees
dt_curr       = 50*ms             # current_clock step length (should be the highest common denominator between tc_start and tr_stop)
tc_stop       = tc_start + tc_dur
current_clock = Clock(dt=dt_curr)


# return the normalised "distance" of the neurons in a circle
def circ_distance(fi):
    if (fi > 0):
        return min(fi,360-fi)
    else:
        return max(fi,fi-360)
    

# calculating the cue currents
currents = lambda i,j: i_cue_amp*exp(-0.5*circ_distance((i-j)*360./NE)**2/i_cue_width**2)

current_e=zeros(NE)
j = i_cue_ang*NE/360.
for i in range(NE): 
    current_e[i]=currents(i,j)

In [5]:
spikes=np.load('/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/spk_arr2.npy')
cell_identity=np.load('/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/cell_identity2.npy')

In [ ]:
eqs_e = '''
dv/dt = (-gl_e*(v-El_e)-g_ext_e*s_ext*(v-E_ampa)-Gee*s_tot*(v-E_nmda)/(1+b*exp(-a*v))-Gie*s_gaba*(v-E_gaba)+i_e)/Cm_e: volt 
ds_ext/dt = -s_ext/t_ampa : 1
ds_gaba/dt = -s_gaba/t_gaba : 1
ds_nmda/dt = -s_nmda/t_nmda+alfa*x*(1-s_nmda) : 1
dx/dt = -x/t_x : 1
s_tot : 1
i_e : amp
'''

eqs_i = '''
dv/dt = (-gl_i*(v-El_i)-g_ext_i*s_ext*(v-E_ampa)-Gei*(v-E_nmda)*s_tot/(1+b*exp(-a*v))-Gii*s_gaba*(v-E_gaba))/Cm_i: volt 
ds_ext/dt = -s_ext/t_ampa : 1
ds_gaba/dt = -s_gaba/t_gaba : 1
s_tot : 1
'''

reset='''
v=Vr_e
x+=1*1
'''

print("Setting up the populations ...")
    
Pe = NeuronGroup(NE, eqs_e, threshold='v>Vth_e', reset='v=Vr_e', refractory=tr_e, clock=simulation_clock, order=2)
Pe.v = El_e
Pe.s_ext = 0
Pe.s_gaba = 0
Pe.s_nmda = 0
Pe.x = 0
Pe.i_e = 0

Pi = NeuronGroup(NI, eqs_i, threshold='v>Vth_i', reset='v=Vr_i', refractory=tr_i, clock=simulation_clock, order=2)
Pi.v = El_i
Pi.s_ext = 0
Pi.s_gaba = 0

# external Poisson input
PG  = PoissonGroup(1000, 1*Hz, clock=simulation_clock)
#PG=SpikeGeneratorGroup(1000,list(cell_identity),spikes*1000*ms)
PGe = PG
PGi = PG

print("Creating connections ...")
w = 1
Cpe = Synapses(PGe, Pe, on_pre='s_ext += w')
Cpe.connect(p=0.5)

Cpi = Synapses(PGi, Pi, on_pre='s_ext += w')
Cpi.connect(p=0.15)

Cie = Synapses(Pi, Pe, on_pre='s_gaba += w')
Cie.connect()
Cii = Synapses(Pi, Pi, on_pre='s_gaba += w')
Cii.connect()


# Calculate NMDA contributions
@network_operation(simulation_clock, when='start')
def update_nmda(simulation_clock):
    fsnmda = rfft(Pe.s_nmda) 
    fstot = fsnmda*fweight
    Pe.s_tot = irfft(fstot, NE)
    Pi.s_tot = fsnmda[0]

'''
# Update stimulus input
@network_operation(current_clock, when='start')
def update_currents(current_clock):
    c_time = current_clock.t
    if (c_time>tc_start and c_time<tc_stop):
        Pe.i_e = current_e
    else:
        Pe.i_e = 0
'''

#simulation_clock.t = -500*ms

# initiating monitors
#spikes_e = FileSpikeMonitor(Pe,spikes_e_file,record=True)
#spikes_i = FileSpikeMonitor(Pi,spikes_i_file,record=True)
M=SpikeMonitor(Pe)
Z=SpikeMonitor(Pi)
Minput = SpikeMonitor(PG)
print("Running ...")
run(simtime)

Setting up the populations ...
Creating connections ...
Running ...


In [ ]:
plot(Minput.t/ms, Minput.i, '.')
plt.show()
plot(M.t/ms, M.i, '.')
plt.show()